In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


#**Data**

In [ ]:
food = pd.read_csv('/content/gdrive/My Drive/Food.csv')
compound = pd.read_csv('/content/gdrive/My Drive/Compound.csv')
nutrient = pd.read_csv('/content/gdrive/My Drive/Nutrient.csv')
content = pd.read_csv('/content/gdrive/My Drive/Content.csv')
health_effect = pd.read_csv('/content/gdrive/My Drive/HealthEffect.csv')
compounds_health_effect = pd.read_csv('/content/gdrive/My Drive/CompoundsHealthEffect.csv')
flavor = pd.read_csv('/content/gdrive/My Drive/Flavor.csv')
compounds_flavor = pd.read_csv('/content/gdrive/My Drive/CompoundsFlavor.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
food = food.replace('NA',np.nan)
food = food[['id', 'name', 'food_group', 'food_subgroup']]
food = food.dropna()

compound = compound.replace('NA',np.nan)
compound = compound[['id', 'name']]
compound = compound.dropna()

nutrient = nutrient.replace('NA',np.nan)
nutrient = nutrient[['id', 'name']]
nutrient = nutrient.dropna()

content = content.replace('NA',np.nan)
content = content[['source_id', 'source_type', 'food_id', 'orig_content', 'orig_unit']]
content_compound = content[(content['source_type'] == 'Compound')]
content_nutrient = content[(content['source_type'] == 'Nutrient')]
content_compound = content_compound.dropna()
content_nutrient = content_nutrient.dropna()

health_effect = health_effect.replace('NA',np.nan)
health_effect = health_effect[['id', 'name']]
health_effect = health_effect.dropna()

compounds_health_effect = compounds_health_effect.replace('NA',np.nan)
compounds_health_effect = compounds_health_effect[['compound_id', 'health_effect_id']]
compounds_health_effect = compounds_health_effect.dropna()

flavor = flavor.replace('NA',np.nan)
flavor = flavor[['id', 'name']]
flavor = flavor.dropna()

compounds_flavor = compounds_flavor.replace('NA',np.nan)
compounds_flavor = compounds_flavor[['compound_id', 'flavor_id']]
compounds_flavor = compounds_flavor.dropna()

In [ ]:
content_compound = content_compound[content_compound.orig_content != 0]
content_nutrient = content_nutrient[content_nutrient.orig_content != 0]
content_compound = content_compound.drop_duplicates(subset=['source_id', 'food_id'])
content_nutrient = content_nutrient.drop_duplicates(subset=['source_id', 'food_id'])

In [ ]:
#Food-Compound
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  food_name = food[food['id'] == food_id]['name'].iloc[0]
  food_group = food[food['id'] == food_id]['food_group'].iloc[0]
  food_subgroup = food[food['id'] == food_id]['food_subgroup'].iloc[0]
  index = content_compound[content_compound['food_id'] == food_id].index
  content_compound.at[index, 'food_name'] = food_name
  content_compound.at[index, 'food_group'] = food_group
  content_compound.at[index, 'food_subgroup'] = food_subgroup

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = content_compound[(content_compound['source_id'] == compound_id)].index
  content_compound.at[index, 'source_name'] = compound_name

In [ ]:
#Food-Nutrient
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  food_name = food[food['id'] == food_id]['name'].iloc[0]
  food_group = food[food['id'] == food_id]['food_group'].iloc[0]
  food_subgroup = food[food['id'] == food_id]['food_subgroup'].iloc[0]
  index = content_nutrient[content_nutrient['food_id'] == food_id].index
  content_nutrient.at[index, 'food_name'] = food_name
  content_nutrient.at[index, 'food_group'] = food_group
  content_nutrient.at[index, 'food_subgroup'] = food_subgroup

for i in range(len(nutrient)):
  nutrient_id = nutrient['id'].iloc[i]
  nutrient_name = nutrient[nutrient['id'] == nutrient_id]['name'].iloc[0]
  index = content_nutrient[(content_nutrient['source_id'] == nutrient_id)].index
  content_nutrient.at[index, 'source_name'] = nutrient_name

In [ ]:
#Compound-Health Effect
for i in range(len(health_effect)):
  health_effect_id = health_effect['id'].iloc[i]
  health_effect_name = health_effect[health_effect['id'] == health_effect_id]['name'].iloc[0]
  index = compounds_health_effect[compounds_health_effect['health_effect_id'] == health_effect_id].index
  compounds_health_effect.at[index, 'health_effect_name'] = health_effect_name

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = compounds_health_effect[compounds_health_effect['compound_id'] == compound_id].index
  compounds_health_effect.at[index, 'compound_name'] = compound_name

In [ ]:
#Compound-Flavor
for i in range(len(flavor)):
  flavor_id = flavor['id'].iloc[i]
  flavor_name = flavor[flavor['id'] == flavor_id]['name'].iloc[0]
  index = compounds_flavor[compounds_flavor['flavor_id'] == flavor_id].index
  compounds_flavor.at[index, 'flavor_name'] = flavor_name

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = compounds_flavor[compounds_flavor['compound_id'] == compound_id].index
  compounds_flavor.at[index, 'compound_name'] = compound_name

In [ ]:
content_compound.to_csv('content_compound.csv')
content_nutrient.to_csv('content_nutrient.csv')
compounds_health_effect.to_csv('compounds_health_effect.csv')
compounds_flavor.to_csv('compounds_flavor.csv')

In [ ]:
from google.colab import files
files.download('content_compound.csv')
files.download('content_nutrient.csv')
files.download('compounds_health_effect.csv')
files.download('compounds_flavor.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>